In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
print(tf.__version__)
print(np.__version__)


2.4.1
1.19.5


In [9]:
file = open("data/data.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

"Skip to main content Epicurious 23 Best Nonalcoholic Drinks (Just Don’t Call Them Mocktails, Please) Recipes & Menus Expert Advice Ingredients Holidays & Events My Saved Recipes Sign In SUBSCRIBE Search Recipes & Menus Expert Advice Ingredients Holidays & Events Video My Saved Recipes Come see what's cooking. Over 50,000 recipes, straight from our test kitchen to yours. Unlimited access for only $40 $30 a year. SUBSCRIBE Non-Alcoholic 23 Best Nonalcoholic Drinks (Just Don’t Call Them Mocktails, "

In [10]:
tokenizer = Tokenizer(oov_token=1)
tokenizer.fit_on_texts([data])
# saving the tokenizer for predict function
pickle.dump(tokenizer, open('data/token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

[140, 4, 276, 277, 25, 8, 35, 21, 36, 31, 141, 50, 51, 91, 142]

In [11]:
vocab_size = len(tokenizer.word_index) + 1
PREDICT_INP=2 # number of words taken as input to predict the next word

In [12]:
sequences = []

for i in range(PREDICT_INP, len(sequence_data)):
    words = sequence_data[i-PREDICT_INP:i+1]
    sequences.append(words)
    
sequences = np.array(sequences)

The Length of sequences are:  1955


array([[140,   4, 276, 277],
       [  4, 276, 277,  25],
       [276, 277,  25,   8],
       [277,  25,   8,  35],
       [ 25,   8,  35,  21],
       [  8,  35,  21,  36],
       [ 35,  21,  36,  31],
       [ 21,  36,  31, 141],
       [ 36,  31, 141,  50],
       [ 31, 141,  50,  51]])

In [15]:
X = []
y = []

for i in sequences:
    X.append(i[0:PREDICT_INP])
    y.append(i[PREDICT_INP])
    
X = np.array(X)
y = np.array(y)

y = to_categorical(y, num_classes=vocab_size)

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=PREDICT_INP))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

2022-07-30 22:44:41.401093: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-30 22:44:41.419243: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-30 22:44:41.463231: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("data/next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

2022-07-30 22:44:55.233872: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-30 22:44:55.355681: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1800000000 Hz


Epoch 1/70
31/31 [==============================] - 13s 229ms/step - loss: 6.5893

Epoch 00001: loss improved from inf to 6.48060, saving model to next_words.h5
Epoch 2/70
31/31 [==============================] - 7s 222ms/step - loss: 6.1258

Epoch 00002: loss improved from 6.48060 to 6.12251, saving model to next_words.h5
Epoch 3/70
31/31 [==============================] - 8s 243ms/step - loss: 6.0229

Epoch 00003: loss improved from 6.12251 to 6.00289, saving model to next_words.h5
Epoch 4/70
31/31 [==============================] - 8s 262ms/step - loss: 5.8908

Epoch 00004: loss improved from 6.00289 to 5.90031, saving model to next_words.h5
Epoch 5/70
31/31 [==============================] - 9s 296ms/step - loss: 5.8048

Epoch 00005: loss improved from 5.90031 to 5.80876, saving model to next_words.h5
Epoch 6/70
31/31 [==============================] - 10s 315ms/step - loss: 5.5749

Epoch 00006: loss improved from 5.80876 to 5.66902, saving model to next_words.h5
Epoch 7/70
31/31 [

Epoch 52/70
31/31 [==============================] - 8s 257ms/step - loss: 0.2022

Epoch 00052: loss improved from 0.28194 to 0.24290, saving model to next_words.h5
Epoch 53/70
31/31 [==============================] - 8s 252ms/step - loss: 0.2345

Epoch 00053: loss did not improve from 0.24290
Epoch 54/70
31/31 [==============================] - 9s 277ms/step - loss: 0.2165

Epoch 00054: loss did not improve from 0.24290
Epoch 55/70
31/31 [==============================] - 8s 267ms/step - loss: 0.2321

Epoch 00055: loss did not improve from 0.24290
Epoch 56/70
31/31 [==============================] - 8s 259ms/step - loss: 0.2177

Epoch 00056: loss improved from 0.24290 to 0.22717, saving model to next_words.h5
Epoch 57/70
31/31 [==============================] - 8s 267ms/step - loss: 0.1707

Epoch 00057: loss improved from 0.22717 to 0.19933, saving model to next_words.h5
Epoch 58/70
31/31 [==============================] - 8s 262ms/step - loss: 0.2312

Epoch 00058: loss did not improv